# Manage runs and experiments with MLFlow

Experiments and runs can be queried using MLflow client in Azure ML. You are not longer in need to use Azure ML specific SDKs to manage anything that happens inside of a training job, allowing you to remove dependencies and create a more seamless transition between local runs and cloud.

MLflow client allows you to:

* Create, delete and search for experiments in a workspace
* Start, stop, cancel and query runs for experiments.
* Track and retrieve metrics, parameters, artifacts and models from runs.

## Prerequisites to run this notebook

In [ ]:
# Ensure you have the dependencies for this notebook
%pip install -r run_history.txt

Import required namespaces:

In [ ]:
import mlflow

In the following notebook, we will explore an example that uses the following naming convention:

In [ ]:
experiment_name = "heart-classifier-sample"
model_name = "heart-classifier"
artifact_path = "pipeline"

To demonstrate how to manage runs, let's create a couple of experiments and runs (3 of them):

In [ ]:
from trainer import train_and_log

In [ ]:
mlflow.set_experiment(experiment_name=experiment_name)
input_data = "https://azuremlexampledata.blob.core.windows.net/data/heart-disease-uci/data/heart.csv"

for run_idx in range(3):
    with mlflow.start_run(run_name=f"{experiment_name}-run-{run_idx}"):
        train_and_log(input_data)

## Getting all the experiments

You can get all the active experiments in the workspace using MLFlow:

> __MLflow 2.0 advisory__: In legacy versions of MLflow (<2.0) use method `list_experiments` instead:

In [ ]:
experiments = mlflow.search_experiments()
for exp in experiments:
    print(exp.name)

If you want to retrieve archived experiments too, then include the option `ViewType.ALL`

In [ ]:
from mlflow.entities import ViewType

experiments = mlflow.search_experiments(view_type=ViewType.ALL)
for exp in experiments:
    print(exp.name)

## Getting an specific experiment

In [ ]:
exp = mlflow.get_experiment_by_name(experiment_name)
print(exp)

## Getting runs inside an experiments

MLflow allows to search runs inside of any experiment. You will need either the experiment ID or the experiment name:

### Getting all the runs

In [ ]:
mlflow.search_runs(exp.experiment_id)

> Notice that `experiment_ids` supports providing an array of experiments, so you can search runs across multiple experiments if required. This may be useful in case you want to compare runs of the same model when it is being logged in different experiments (by different people, different project iterations, etc). You can also use `search_all_experiments=True` if you want to search across all the experiments in the workspace.

### Filtering and ordering runs

By default, experiments are ordered descending by `start_time`, which is the time the experiment was queue in Azure ML. However, you can change this default by using the parameter `order_by`.

In [ ]:
mlflow.search_runs(exp.experiment_id, order_by=["start_time DESC"], max_results=2)

You can also search by metrics to know which run generated the best results:

> **Notes:** Expressions containing `metrics.*` in the parameter `order_by` is not supported by the moment. Please use `order_values` method from Pandas as shown in the next example.

In [ ]:
mlflow.search_runs(exp.experiment_id).sort_values("metrics.accuracy", ascending=False)

You can also look for a run with an specific combination in the hyperparameters:

In [ ]:
mlflow.search_runs(
    exp.experiment_id, filter_string="params.num_boost_round='100'", max_results=2
)

You can also filter experiment by status. This is useful to find runs that are running, completed, canceled or failed. In MLflow, `status` is an `attribute`, so we can access this value using the expression `attributes.status`. The following table shows the possible values:

| Azure ML Job status | MLFlow's `attributes.status` | Meaning |
| :-: | :-: | :- |
| Not started | `SCHEDULED` | The job/run was just registered in Azure ML but it has processed it yet. |
| Queue | `SCHEDULED` | The job/run is scheduled for running, but it hasn't started yet. |
| Preparing | `SCHEDULED` | The job/run has not started yet, but a compute has been allocated for the execution and it is on building state. |
| Running | `RUNNING` | The job/run is currently under active execution. |
| Completed | `FINISHED` | The job/run has completed without errors. |
| Failed | `FAILED` | The job/run has completed with errors. |
| Canceled | `KILLED` | The job/run has been canceled or killed by the user/system. |


> **Notes:** Expressions containing `attributes.status` in the parameter `filter_string` are not support at the moment. Please use Pandas filtering expressions as shown in the next example.

In [ ]:
runs = mlflow.search_runs(exp.experiment_id)
runs[runs.status == "FINISHED"]

By default, mlflow returns runs as a pandas Dataframe. You can get Python objects if required, which may be useful to get details about them:

In [ ]:
runs = mlflow.search_runs(
    exp.experiment_id,
    filter_string="params.num_boost_round='100'",
    output_format="list",
)

For instance, you can get the last run matching the search criteria by:

In [ ]:
last_run = runs[-1]
print(last_run)

## Getting metrics, params, artifacts and models from a run

Once you have identified the run you are interested in, you can get details about it to further explore.

### Metrics

In [ ]:
last_run.data.metrics

### Parameters

In the same way, you can get all the parameters of the run:

In [ ]:
last_run.data.params

### Artifacts

For those runs that log artifacts, you can also list them. However, that can't be done from the run object itself, but you need an MLflow client object:

In [ ]:
client = mlflow.tracking.MlflowClient()
client.list_artifacts(last_run.info.run_id)

As you can see in this example, three artifacts are availble in the run:

* `feature_importance_weight.json` -> the feature importance of the model we created.
* `feature_importance_weight.png` -> a plot of the feature importance mentioned above, stored as an image.
* `classifier`, the path where the model is stored. Note that this artifact is a directory.

You can download any artifact using the method `download_artifact`

In [ ]:
file_path = mlflow.artifacts.download_artifacts(
    run_id=last_run.info.run_id,
    artifact_path="feature_importance_weight.png"
)

> __MLflow 2.0 advisory__: In legacy versions of MLflow (<2.0), use the method `MlflowClient.download_artifacts()` instead.

Since the artifact is an image, we can display it in the following way:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as img

image = img.imread(file_path)
plt.imshow(image)
plt.show()

### Models

Models can also be logged in the run and then retrieved directly from it. To retrieve it, you need to know the artifact's path where it is stored. The method `list_artifacats` can be used to find artifacts that are representing a model. Those that are folder, for instance.

You can download a model them by indicating the path where the model is stored using the `download_artifact` method:

In [ ]:
model_local_path = mlflow.artifacts.download_artifacts(
    run_id=last_run.info.run_id, 
    artifact_path=artifact_path
)

You can then load the model from the local path using MLflow:

In [ ]:
model = mlflow.sklearn.load_model(model_local_path)

MLflow also allows you to skip the download path and directly reference the model from the run. You can achieve this using the following URI format:

In [ ]:
model = mlflow.sklearn.load_model(f"runs:/{last_run.info.run_id}/{artifact_path}")

> This will download the model to a temporary folder and load the model from there. Note that loading a model requires you to have all its dependencies already installed in you environment. MLflow will still check if versions installed are present before loading.